In [2]:
import cv2
import mediapipe as mp
import numpy as np
import os
import csv
from matplotlib import pyplot as plt
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_holistic = mp.solutions.holistic
mp_drawing_styles = mp.solutions.drawing_styles


In [4]:
# VIDEO FEED
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('Mediapipe Feed', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [59]:
cap = cv2.VideoCapture(0)
# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640,  480))
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    out.write(frame)
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'):
        break
# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()

In [7]:
landmarks = ['class']
for val in range(1,33+1):
    landmarks += ['x{}'.format(val),'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

In [6]:
with open('coords.csv',mode='w',newline='') as f:
    csv_writer = csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [7]:
def export_landmarks(results,action):
    try:
        keypoints = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
        keypoints.insert(0,action)

        with open('coords.csv',mode='a',newline='') as f:
            csv_writer = csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        pass



In [8]:
# For webcam input:
cap = cv2.VideoCapture('output.avi')
with mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as holistic:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = holistic.process(image)

    # Draw landmark annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.face_landmarks,
        mp_holistic.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing_styles
        .get_default_face_mesh_contours_style())
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles
        .get_default_pose_landmarks_style())
    # Flip the image horizontally for a selfie-view display.


    k=cv2.waitKey(1)
    if k == 117:
      export_landmarks(results,'up')
    if k == 100:
      export_landmarks(results,'down')
    cv2.imshow('MediaPipe Holistic', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()
cv2.destroyAllWindows()

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [16]:
df = pd.read_csv('coordsFinal.csv')

In [17]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,up,0.599003,0.177474,-0.437339,0.999987,0.606967,0.161064,-0.420176,0.999957,0.612775,...,0.267559,0.919380,0.603511,0.919829,0.017353,0.953921,0.538112,0.924975,0.103684,0.964305
1,up,0.571595,0.149262,-0.492643,0.999994,0.579862,0.132655,-0.468533,0.999980,0.583889,...,0.217522,0.827504,0.662186,0.977665,0.062819,0.983946,0.492033,0.981400,0.016847,0.983339
2,up,0.571757,0.151450,-0.475689,0.999993,0.580026,0.133887,-0.451895,0.999977,0.583996,...,0.218432,0.816996,0.655882,0.970831,0.049677,0.982327,0.498544,0.979039,0.011513,0.981558
3,up,0.571780,0.151599,-0.471709,0.999993,0.580091,0.134050,-0.447993,0.999976,0.584055,...,0.215975,0.814146,0.655657,0.971157,0.048230,0.981686,0.498623,0.979080,0.011098,0.980907
4,up,0.571781,0.151665,-0.471722,0.999993,0.580121,0.134150,-0.448010,0.999976,0.584083,...,0.210996,0.812430,0.655569,0.971219,0.045871,0.981400,0.498518,0.979287,0.007286,0.980642


In [18]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
1111,up,0.580477,0.141967,-0.503158,0.999990,0.587906,0.126540,-0.478903,0.999969,0.593171,...,0.180483,0.792605,0.643594,0.964724,0.038171,0.971817,0.502381,0.969185,-0.021153,0.969793
1112,up,0.582632,0.141659,-0.509767,0.999991,0.589916,0.126185,-0.484743,0.999971,0.595251,...,0.193561,0.787485,0.643520,0.964477,0.083993,0.971641,0.503081,0.969179,-0.006987,0.969696
1113,up,0.583471,0.141542,-0.506625,0.999991,0.590757,0.125854,-0.482053,0.999972,0.596130,...,0.205824,0.783855,0.643479,0.964906,0.070337,0.971365,0.503074,0.969167,0.009946,0.969378
1114,up,0.585746,0.141461,-0.514426,0.999992,0.593175,0.125637,-0.489996,0.999974,0.598345,...,0.208681,0.782221,0.643580,0.964701,0.068721,0.971498,0.503088,0.969215,0.017426,0.969495
1115,up,0.588085,0.141327,-0.511495,0.999992,0.595892,0.125293,-0.487294,0.999975,0.600836,...,0.207694,0.783229,0.643736,0.964603,0.063613,0.971291,0.502544,0.969228,0.020998,0.969533


In [19]:
df[df['class']=='up']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,up,0.599003,0.177474,-0.437339,0.999987,0.606967,0.161064,-0.420176,0.999957,0.612775,...,0.267559,0.919380,0.603511,0.919829,0.017353,0.953921,0.538112,0.924975,0.103684,0.964305
1,up,0.571595,0.149262,-0.492643,0.999994,0.579862,0.132655,-0.468533,0.999980,0.583889,...,0.217522,0.827504,0.662186,0.977665,0.062819,0.983946,0.492033,0.981400,0.016847,0.983339
2,up,0.571757,0.151450,-0.475689,0.999993,0.580026,0.133887,-0.451895,0.999977,0.583996,...,0.218432,0.816996,0.655882,0.970831,0.049677,0.982327,0.498544,0.979039,0.011513,0.981558
3,up,0.571780,0.151599,-0.471709,0.999993,0.580091,0.134050,-0.447993,0.999976,0.584055,...,0.215975,0.814146,0.655657,0.971157,0.048230,0.981686,0.498623,0.979080,0.011098,0.980907
4,up,0.571781,0.151665,-0.471722,0.999993,0.580121,0.134150,-0.448010,0.999976,0.584083,...,0.210996,0.812430,0.655569,0.971219,0.045871,0.981400,0.498518,0.979287,0.007286,0.980642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111,up,0.580477,0.141967,-0.503158,0.999990,0.587906,0.126540,-0.478903,0.999969,0.593171,...,0.180483,0.792605,0.643594,0.964724,0.038171,0.971817,0.502381,0.969185,-0.021153,0.969793
1112,up,0.582632,0.141659,-0.509767,0.999991,0.589916,0.126185,-0.484743,0.999971,0.595251,...,0.193561,0.787485,0.643520,0.964477,0.083993,0.971641,0.503081,0.969179,-0.006987,0.969696
1113,up,0.583471,0.141542,-0.506625,0.999991,0.590757,0.125854,-0.482053,0.999972,0.596130,...,0.205824,0.783855,0.643479,0.964906,0.070337,0.971365,0.503074,0.969167,0.009946,0.969378
1114,up,0.585746,0.141461,-0.514426,0.999992,0.593175,0.125637,-0.489996,0.999974,0.598345,...,0.208681,0.782221,0.643580,0.964701,0.068721,0.971498,0.503088,0.969215,0.017426,0.969495


In [20]:
X = df.drop('class',axis=1) #features
y = df['class'] #target

In [21]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=1234)

In [22]:
y_test

266       up
1039      up
62        up
484     down
686     down
        ... 
516       up
766     down
718       up
159     down
271       up
Name: class, Length: 335, dtype: object

### Train Machine learning classification Model

In [23]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression,RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [24]:
pipelines = {
    'lr':make_pipeline(StandardScaler(),LogisticRegression()),
    'rc':make_pipeline(StandardScaler(),RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(),RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(),GradientBoostingClassifier())
}

In [25]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train,y_train)
    fit_models[algo] = model
    

In [26]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [27]:
fit_models['rc'].predict(X_test)

array(['up', 'up', 'up', 'down', 'down', 'up', 'down', 'up', 'up', 'down',
       'up', 'up', 'up', 'down', 'up', 'up', 'down', 'up', 'down', 'down',
       'up', 'up', 'up', 'up', 'down', 'up', 'up', 'up', 'down', 'up',
       'down', 'up', 'up', 'down', 'up', 'down', 'down', 'down', 'up',
       'down', 'up', 'up', 'down', 'up', 'up', 'down', 'down', 'up',
       'down', 'up', 'down', 'up', 'down', 'down', 'up', 'up', 'up', 'up',
       'up', 'up', 'down', 'down', 'up', 'down', 'up', 'down', 'up', 'up',
       'up', 'up', 'up', 'up', 'down', 'up', 'down', 'down', 'down', 'up',
       'up', 'down', 'up', 'down', 'up', 'down', 'down', 'down', 'up',
       'down', 'up', 'up', 'up', 'up', 'up', 'down', 'up', 'down', 'up',
       'down', 'down', 'up', 'up', 'down', 'up', 'up', 'down', 'up',
       'down', 'up', 'down', 'up', 'up', 'up', 'up', 'up', 'up', 'down',
       'up', 'up', 'up', 'up', 'down', 'down', 'up', 'up', 'down', 'up',
       'down', 'down', 'down', 'up', 'up', 'up', 'up', 

### Evaluate and Serialize Model

In [3]:
from sklearn.metrics import accuracy_score, precision_score, recall_score # Accuracy Metrics
import pickle

In [29]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo,accuracy_score(y_test.values,yhat),
    precision_score(y_test.values,yhat,average='binary',pos_label='up'),
    recall_score(y_test.values,yhat,average='binary',pos_label='up'))


lr 1.0 1.0 1.0
rc 1.0 1.0 1.0
rf 1.0 1.0 1.0
gb 1.0 1.0 1.0


In [30]:
yhat = fit_models['rf'].predict(X_test)

In [31]:
yhat[:10]

array(['up', 'up', 'up', 'down', 'down', 'up', 'down', 'up', 'up', 'down'],
      dtype=object)

In [32]:
y_test[:10]

266       up
1039      up
62        up
484     down
686     down
33        up
423     down
99        up
1107      up
768     down
Name: class, dtype: object

In [33]:
with open('deadlift.pkl','wb') as f:
    pickle.dump(fit_models['rf'],f)


### Make detections with the Model

In [11]:
with open('deadlift.pkl','rb') as f:
    model = pickle.load(f)

In [12]:
landmarks = ['class']
for val in range(1,33+1):
    landmarks += ['x{}'.format(val),'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

In [5]:
np.array([1.,0.]).argmax()


0

In [4]:
# For webcam input:
counter = 0
current_stage = ''
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = holistic.process(image)

    # Draw landmark annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    # mp_drawing.draw_landmarks(
    #     image,
    #     results.face_landmarks,
    #     mp_holistic.FACEMESH_CONTOURS,
    #     landmark_drawing_spec=None,
    #     connection_drawing_spec=mp_drawing_styles
    #     .get_default_face_mesh_contours_style())
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles
        .get_default_pose_landmarks_style())
    
    try:
        row = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
        X = pd.DataFrame([row],columns=landmarks[1:])
        body_language_class = model.predict(X)[0]
        body_language_prob = model.predict_proba(X)[0]
        print(body_language_class,body_language_prob)

        if body_language_class == 'down' and body_language_prob[body_language_prob.argmax()] >= 0.7:
            current_stage = 'down'
            print("huuuuuu")
        elif current_stage == 'down' and body_language_class == 'up' and body_language_prob[body_language_prob.argmax()] >= 0.7:
            current_stage = 'up'
            counter += 1
            print(counter)
            print("this was hit!")
        
        #Get status box
        cv2.rectangle(image,(0,0),(250,60),(245,117,16),-1)

        #Display class
        cv2.putText(image,'CLASS'
            ,(95,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
        cv2.putText(image,body_language_class.split(' ')[0]
        , (90,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

        #Display Probability
        cv2.putText(image,'PROB',(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
        cv2.putText(image,str(round(body_language_prob[np.argmax(body_language_prob)],2))
        ,(10,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

        #Display Probability
        cv2.putText(image,'COUNT',(180,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
        cv2.putText(image,str(counter),(175,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

    except Exception as e:
        print('error',e)
        pass

    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Holistic',image ) #cv2.flip(image, 1)
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()
cv2.destroyAllWindows()

error 'NoneType' object has no attribute 'landmark'
error name 'pd' is not defined
error name 'pd' is not defined
error name 'pd' is not defined
error name 'pd' is not defined
error name 'pd' is not defined
error name 'pd' is not defined
error name 'pd' is not defined
error name 'pd' is not defined
error name 'pd' is not defined
error name 'pd' is not defined
error name 'pd' is not defined
error name 'pd' is not defined
error name 'pd' is not defined
error name 'pd' is not defined
error name 'pd' is not defined
error name 'pd' is not defined
error name 'pd' is not defined
error name 'pd' is not defined
error name 'pd' is not defined
error name 'pd' is not defined
error name 'pd' is not defined
error name 'pd' is not defined
error name 'pd' is not defined
error name 'pd' is not defined
error name 'pd' is not defined
error name 'pd' is not defined
error name 'pd' is not defined
error name 'pd' is not defined
error name 'pd' is not defined
error name 'pd' is not defined
error name 'pd' is

In [3]:
# Raw Package
import numpy as np
import pandas as pd

#Data Source
import yfinance as yf

#Data viz
# import plotly.graph_objs as go

# Get Bitcoin data
data = yf.download(tickers='BTC-USD', period = '22h', interval = '15m')

# #declare figure
# fig = go.Figure()

# #Candlestick
# fig.add_trace(go.Candlestick(x=data.index,
#                 open=data['Open'],
#                 high=data['High'],
#                 low=data['Low'],
#                 close=data['Close'], name = 'market data'))

# # Add titles
# fig.update_layout(
#     title='Bitcoin live share price evolution',
#     yaxis_title='Bitcoin Price (kUS Dollars)')

# # X-Axes
# fig.update_xaxes(
#     rangeslider_visible=True,
#     rangeselector=dict(
#         buttons=list([
#             dict(count=15, label="15m", step="minute", stepmode="backward"),
#             dict(count=45, label="45m", step="minute", stepmode="backward"),
#             dict(count=1, label="HTD", step="hour", stepmode="todate"),
#             dict(count=6, label="6h", step="hour", stepmode="backward"),
#             dict(step="all")
#         ])
#     )
# )

# #Show
# fig.show()
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-12-27 12:45:00,16838.837891,16841.181641,16826.906250,16826.906250,16826.906250,77481984
2022-12-27 13:00:00,16827.451172,16841.763672,16824.796875,16841.763672,16841.763672,39220224
2022-12-27 13:15:00,16841.142578,16843.919922,16837.585938,16841.921875,16841.921875,0
2022-12-27 13:30:00,16842.421875,16843.060547,16834.964844,16835.197266,16835.197266,4617216
2022-12-27 13:45:00,16834.140625,16834.140625,16826.273438,16829.707031,16829.707031,18203648
...,...,...,...,...,...,...
2022-12-28 08:45:00,16658.220703,16658.220703,16648.597656,16655.722656,16655.722656,23747584
2022-12-28 09:00:00,16654.916016,16654.916016,16642.275391,16642.275391,16642.275391,0
2022-12-28 09:15:00,16642.322266,16647.416016,16642.322266,16646.595703,16646.595703,16797696


: 